### Runner

The `Runner` class appears to be a new module in future version of FastAI. Its goal is to create a single class that "*has access to everything and can change anything at any time*". This allows the `Callback` class to be simplified.

It utilizes a couple of helper functions: `camel2snake` and `listify` as shown below:

#### camel2snake

In [21]:
import re

def camel2snake(name):
    """
    Helper function that simply turns such string names like "AbcDefGh" into "abc_def_gh"
    """
    _camel_re1 = re.compile('(.)([A-Z][a-z]+)')
    _camel_re2 = re.compile('([a-z0-9])([A-Z])')
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

In [5]:
print(f'AbcDef --> {camel2snake("AbcDef")}')

AbcDef --> abc_def


#### listify

Returns input as list of "stuff"

In [2]:
from typing import *

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [13]:
# test cases for listify
print(listify(None))
print(listify("Tom"))
print(listify(34))
print(listify([1,"43",'Tom']))
print(listify(iter([1,2,3])))

[]
['Tom']
[34]
[1, '43', 'Tom']
[1, 2, 3]


#### Callback (simplified)

Note that the new `Callback` class is much more minimalistic than what's implemented in v1 of FastAI!

In [14]:
class Callback():
    _order=0 # used to sort order of callback to execute, 0 being the base layer parent Callback class
    def set_runner(self, run): self.run=run
    def __getattr__(self, k): return getattr(self.run, k)  # reaches into runner to get attribute k
    
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')
    
    def __call__(self, cb_name):  # check if the callback specified by cb_name exists
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

#### TrainEvalCallback

This is simple callback that toggles the model between training and eval modes

In [15]:
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

#### Custom exceptions

This is a rather wacky idea to use exceptions as a flow control mechanism in our training loop. The idea here is to define three-levels of callback mechanisms:
- level 0: at cancel train level
- leve1 1: at cancel epoch level
- level 2: at cancel batch level

This provides all other callbacks with the option to raise these exceptions. For example: 
```
def MyCallback(Callback): 
    ...
    if condition:
        raise CancelTrainException()
``` 
Where the `condition` could be the condition for early stopping.

In [38]:
class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

#### Runner
The `Runner` class is an encapsulating wrapper around callbacks, dataset, model, loss, and optimizer. It is actually a objectified version of our "*infinitely customizable training loop*" that we studied before.

In [32]:
class Runner():
    
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            # set self.(cb.name) to the callback object (i.e. self.train_eval = TrainEvalCallback()
            cb = cbf()            
            setattr(self, cb.name, cb)  
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs

    # this is the method called whenever you see self(callback_name)
    def __call__(self, cb_name):
        # default to false
        res = False
        # sort the order of callbacks to execute 
        for cb in sorted(self.cbs, key=lambda x: x._order): 
            res = cb(cb_name) or res
        return res

    # pass-along property methods that makes it easier to access learner properties from runner object
    @property
    def opt(self):       return self.learn.opt
    @property
    def model(self):     return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data

    # single-batch rountine, called in the all_batches method
    def one_batch(self, xb, yb):
        try:
            self.xb,self.yb = xb,yb
            self('begin_batch')  # ex. call the "begin_batch" callback
            self.pred = self.model(self.xb)
            self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb)
            self('after_loss')
            if not self.in_train: return # stop here if in eval mode as you don't need to backprop & update
            self.loss.backward()
            self('after_backward')
            self.opt.step()
            self('after_step')
            self.opt.zero_grad()
        # call after_cancel_batch callback if CancelBatchException occurs
        except CancelBatchException: self('after_cancel_batch')  
        finally: self('after_batch')

    def all_batches(self, dl):
        self.iters = len(dl)
        try:
            for xb,yb in dl: self.one_batch(xb, yb)
        # call after_cancel_epoch callback if CancelEpochException occurs
        except CancelEpochException: self('after_cancel_epoch')

    def fit(self, epochs, learn):
        # initialization
        self.epochs,self.learn,self.loss = epochs,learn,tensor(0.)

        try:
            # pass the runner object to all the callbacks (note this is where the set_runner method is called)
            for cb in self.cbs: cb.set_runner(self)
            self('begin_fit')
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)
                with torch.no_grad(): 
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                self('after_epoch')
        # call after_cancel_epoch callback if CancelEpochException occurs
        except CancelTrainException: self('after_cancel_train')
        # cleanup
        finally:
            self('after_fit')
            self.learn = None